In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install py7zr

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.4 MB/s eta 0:00:00


In [ ]:
import py7zr

with py7zr.SevenZipFile('/content/drive/MyDrive/data.7z', mode='r') as archive:
    archive.extractall(path='/content/data')


In [ ]:
# Image Preparation
from image_processor import ImageProcessor

csv_path = "/content/New_labels.csv"
image_folder = "/content/data/data/"

processor = ImageProcessor(csv_path, image_folder)
img_data, labels = processor.process_images()




In [ ]:
# Data Preparation
from data_preprocessor import DataPreprocessor
preprocessor = DataPreprocessor(img_data, labels)
train_images, val_images, train_labels, val_labels = preprocessor.preprocess_data()


In [ ]:
# Model Training
from speaker_agent import SpeakerAgent
agent = SpeakerAgent(train_images, train_labels, val_images, val_labels)
model = agent.create_model()


Epoch 1/30
400/400 [==============================] - 38s 61ms/step - loss: 3.8475 - accuracy: 0.1080 - val_loss: 9.3791 - val_accuracy: 0.0059
Epoch 2/30
400/400 [==============================] - 22s 54ms/step - loss: 2.2665 - accuracy: 0.3089 - val_loss: 2.0228 - val_accuracy: 0.3384
Epoch 3/30
400/400 [==============================] - 24s 60ms/step - loss: 1.7265 - accuracy: 0.4268 - val_loss: 1.1837 - val_accuracy: 0.6047
Epoch 4/30
400/400 [==============================] - 21s 53ms/step - loss: 1.4372 - accuracy: 0.5144 - val_loss: 0.8727 - val_accuracy: 0.7266
Epoch 5/30
400/400 [==============================] - 23s 59ms/step - loss: 1.2520 - accuracy: 0.5698 - val_loss: 0.6900 - val_accuracy: 0.7969
Epoch 6/30
400/400 [==============================] - 24s 61ms/step - loss: 1.0932 - accuracy: 0.6295 - val_loss: 0.6277 - val_accuracy: 0.8372
Epoch 7/30
400/400 [==============================] - 24s 60ms/step - loss: 1.0032 - accuracy: 0.6519 - val_loss: 0.4228 - val_accuracy:

In [ ]:
# Define the path to save the model
from tensorflow.keras.models import load_model
model_path = 'speaker_agent.h5'

model.save(model_path)
loaded_model = load_model(model_path)

# Game Round 1

In [ ]:
# Usage
from sklearn.preprocessing import LabelEncoder
from game_environment_speaker import GameEnvironmentSpeaker
import numpy as np
import pandas as pd
# Load the Excel file into a pandas DataFrame
data = pd.read_csv("/content/New_labels.csv")

# Extract the filenames and labels into separate arrays
filenames = data['Filenames'].values
shapes = data['labels'].values
shapes = np.array(shapes)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(shapes)
file_path = "/content/test_image.xlsx"
test_img_dir = "/content/test/"


game_env_speaker = GameEnvironmentSpeaker(file_path, test_img_dir, loaded_model, label_encoder)
test_img, target_label, true_labels = game_env_speaker.preprocess_data()
predicted_labels, speaker_reward, sin_plu_distinct_reward = game_env_speaker.play_game(test_img, target_label)


1/1 [==============================] - 0s 18ms/step


In [ ]:
from listener_agent import ListenerAgent

file_path = "/content/test_image.xlsx"
test_data = pd.read_excel(file_path)
test_img = test_data['Filepath']
target_label = test_data['shapes']
test_img_dir = "/content/test/"



In [ ]:

from game_env_listener import GameEnvironmentListener
# Create an instance of the GameEnvironmentListener class
game_env_listener = GameEnvironmentListener(data_path='/content/data/data/', filenames=filenames, shapes=shapes)

# Call the play_game method
test_img_dir = "/content/test/"
test_img = test_data['Filepath']
predicted_labels = predicted_labels

matched_images, predicted_labels_listener, listener_reward = game_env_listener.play_game(test_img_dir, test_img, predicted_labels)



# Round 1 reward Accuracy

In [10]:
speaker_reward_percentage = speaker_reward/len(test_img)
listener_reward_percentage = listener_reward/len(test_img)
sin_plu_distinct_reward_percentage = sin_plu_distinct_reward/len(test_img)
print(speaker_reward_percentage)
print(listener_reward_percentage)
print(sin_plu_distinct_reward_percentage)

0.1375
1.0
0.9625


# Game round 2

In [ ]:
from policy_updater import ModelUpdater
import tensorflow as tf

# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, speaker_reward_percentage, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_1.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)


Epoch 1/30
400/400 [==============================] - 28s 60ms/step - loss: 0.2642 - accuracy: 0.9128 - val_loss: 0.0354 - val_accuracy: 0.9928
Epoch 2/30
400/400 [==============================] - 24s 59ms/step - loss: 0.2517 - accuracy: 0.9152 - val_loss: 0.0374 - val_accuracy: 0.9947
Epoch 3/30
400/400 [==============================] - 22s 55ms/step - loss: 0.2228 - accuracy: 0.9263 - val_loss: 0.0449 - val_accuracy: 0.9916
Epoch 4/30
400/400 [==============================] - 23s 57ms/step - loss: 0.2125 - accuracy: 0.9297 - val_loss: 0.0599 - val_accuracy: 0.9887
Epoch 5/30
400/400 [==============================] - 22s 55ms/step - loss: 0.2032 - accuracy: 0.9356 - val_loss: 0.1400 - val_accuracy: 0.9603
Epoch 6/30
400/400 [==============================] - 24s 59ms/step - loss: 0.2001 - accuracy: 0.9341 - val_loss: 0.0395 - val_accuracy: 0.9934
Epoch 7/30
400/400 [==============================] - 21s 53ms/step - loss: 0.1986 - accuracy: 0.9333 - val_loss: 0.0343 - val_accuracy:

In [ ]:
game_env_speaker = GameEnvironmentSpeaker(file_path, test_img_dir, loaded_model, label_encoder)
test_img, target_label, true_labels = game_env_speaker.preprocess_data()
predicted_labels, speaker_reward, sin_plu_distinct_reward = game_env_speaker.play_game(test_img, target_label)


1/1 [==============================] - 0s 20ms/step


In [ ]:

# Create an instance of the GameEnvironmentListener class
game_env_listener = GameEnvironmentListener(data_path='/content/data/data/', filenames=filenames, shapes=shapes)
matched_images, predicted_labels_listener, listener_reward = game_env_listener.play_game(test_img_dir, test_img, predicted_labels)



In [12]:
speaker_reward_percentage = speaker_reward/len(test_img)
listener_reward_percentage = listener_reward/len(test_img)
sin_plu_distinct_reward_percentage = sin_plu_distinct_reward/len(test_img)
print(speaker_reward_percentage)
print(listener_reward_percentage)
print(sin_plu_distinct_reward_percentage)

0.15
1.0
0.9125


# Game Round 3

In [ ]:
# Create an instance of the ModelUpdater class
model_updater = ModelUpdater(weights_path='speaker_agent_updated_1.h5')

# Call the update_model method
model, lr = model_updater.update_model(loaded_model, speaker_reward_percentage, train_images, train_labels, val_images, val_labels, max_epochs=30)

# Save the updated model
model_path = 'speaker_agent_updated_2.h5'
model.save(model_path)

# Load the updated model
loaded_model = tf.keras.models.load_model(model_path)


Epoch 1/30
400/400 [==============================] - 27s 55ms/step - loss: 0.1341 - accuracy: 0.9546 - val_loss: 0.0297 - val_accuracy: 0.9962
Epoch 2/30
400/400 [==============================] - 23s 59ms/step - loss: 0.1222 - accuracy: 0.9597 - val_loss: 0.0626 - val_accuracy: 0.9847
Epoch 3/30
400/400 [==============================] - 24s 59ms/step - loss: 0.1299 - accuracy: 0.9580 - val_loss: 0.0506 - val_accuracy: 0.9894
Epoch 4/30
400/400 [==============================] - 24s 60ms/step - loss: 0.1302 - accuracy: 0.9559 - val_loss: 0.0351 - val_accuracy: 0.9947
Epoch 5/30
400/400 [==============================] - 22s 55ms/step - loss: 0.1396 - accuracy: 0.9530 - val_loss: 0.0392 - val_accuracy: 0.9931
Epoch 6/30
400/400 [==============================] - 24s 61ms/step - loss: 0.1247 - accuracy: 0.9592 - val_loss: 0.0525 - val_accuracy: 0.9891
Epoch 7/30
400/400 [==============================] - 24s 61ms/step - loss: 0.1280 - accuracy: 0.9578 - val_loss: 0.0360 - val_accuracy:

In [ ]:

game_env_speaker = GameEnvironmentSpeaker(file_path, test_img_dir, loaded_model, label_encoder)
test_img, target_label, true_labels = game_env_speaker.preprocess_data()
predicted_labels, speaker_reward, sin_plu_distinct_reward = game_env_speaker.play_game(test_img, target_label)


1/1 [==============================] - 0s 28ms/step


In [ ]:

# Create an instance of the GameEnvironmentListener class
game_env_listener = GameEnvironmentListener(data_path='/content/data/data/', filenames=filenames, shapes=shapes)
matched_images, predicted_labels_listener, listener_reward = game_env_listener.play_game(test_img_dir, test_img, predicted_labels)



In [14]:
speaker_reward_percentage = speaker_reward/len(test_img)
listener_reward_percentage = listener_reward/len(test_img)
sin_plu_distinct_reward_percentage = sin_plu_distinct_reward/len(test_img)
print(speaker_reward_percentage)
print(listener_reward_percentage)
print(sin_plu_distinct_reward_percentage)

0.1625
1.0
0.925
